# Entanglement & Bell States — Detailed Notes (Session 3)
**Course:** CS490/5590 — Quantum Computing Applications in Data Science, AI, & Deep Learning  
**Instructor:** Luke Miller  
**Level:** Graduate / Advanced Undergraduate  

> **Purpose of these notes.** A self-contained reference for Session 3. We expand the slide bullets into definitions, proofs, derivations, Qiskit snippets, and mini-exercises (solutions sketched at the end).

---

## Session Overview  

**Objective.** Understand quantum entanglement as a fundamental resource; create, measure, and reason about Bell states; connect to teleportation, super-dense coding, and AI/DS use cases.  

**Agenda.**
1. Quick recap of circuits & gates  
2. Definition and character of entanglement  
3. EPR paradox & non-locality  
4. Bell inequalities (CHSH)  
5. Creating & measuring Bell states  
6. Teleportation and super-dense coding protocols  
7. Entanglement in quantum-enhanced ML  
8. Visualisation tools & metrics  
9. Q&A  

---

## 0) Recap from Session 2  

| Concept | One-line reminder |
|---------|-------------------|
| **Circuits** | Ordered products of unitaries acting on qubits |
| **Single-qubit gates** | \(X,Y,Z,H,S,T,R_\alpha(\theta)\) = rotations on Bloch sphere |
| **Multi-qubit gates** | CNOT/CZ (controlled ops) create entanglement |
| **Composition** | Tensor products & matrix multiplication (right→left) |
| **Visualisation** | Circuit diagrams (`qc.draw`), Bloch sphere for single qubit |

**Bridge to today.**  Multi-qubit gates + superposition ⇒ **entanglement**, yielding correlations impossible classically.

---

## 1) What *is* quantum entanglement?

**Definition (informal).** A multi-qubit state is *entangled* if it cannot be written as a product \(|\psi\rangle=\bigotimes_i|\phi_i\rangle\).  

**Formal test (2 qubits).** State \(|\Psi\rangle\) is separable ⇔ its coefficient matrix is rank 1. Any rank > 1 state is entangled.

Example non-separable state  
\[
|\Phi^{+}\rangle=\frac{1}{\sqrt2}(|00\rangle+|11\rangle).
\]

**Key properties**
- *Perfect* correlations (or anti-correlations) upon measurement.  
- No faster-than-light signalling (no-signalling theorem).  
- Fragile: decoherence quickly degrades entanglement.

**Why it matters.** Entanglement is the “fuel” behind teleportation, super-dense coding, quantum error-correction, some algorithmic speedups, and quantum cryptography.

---

## 2) The EPR paradox & non-locality  

**EPR (1935) argument.**  
1. Consider a pair of particles in an entangled state.  
2. Measure an observable (e.g., position) on one ⇒ instantly determine the other.  
3. Likewise for momentum.  
4. Appears to defeat the Heisenberg uncertainty principle *if* reality is local & deterministic.

**Resolution.** Quantum mechanics is *non-local* in correlations but still respects relativistic causality—no information travels faster than light.

> **AI/DS implication.** Entanglement allows distributed quantum subroutines that share non-classical correlations without high bandwidth.

---

## 3) Bell inequalities (CHSH)

**Bell (1964).** Provides testable inequalities to separate quantum predictions from all local hidden-variable theories.

### CHSH expression
\[
S = |E(a,b) + E(a,b') + E(a',b) - E(a',b')| \le 2 \quad \text{(local realism)}
\]
Quantum mechanics permits \(S_{\max}=2\sqrt2\) (Tsirelson bound).

**Experiment.** Choose measurement settings \(a,a'\) for Alice and \(b,b'\) for Bob on a shared Bell pair. Repeated runs violate \(S\le2\). Loophole-free experiments (2015–) confirm quantum mechanics.

---

## 4) Bell states (maximally entangled basis)

\[
\begin{aligned}
|\Phi^{+}\rangle &= \frac{|00\rangle+|11\rangle}{\sqrt2}, &
|\Phi^{-}\rangle &= \frac{|00\rangle-|11\rangle}{\sqrt2}, \\
|\Psi^{+}\rangle &= \frac{|01\rangle+|10\rangle}{\sqrt2}, &
|\Psi^{-}\rangle &= \frac{|01\rangle-|10\rangle}{\sqrt2}. \\
\end{aligned}
\]

- Orthogonal; form a complete basis for 2-qubit Hilbert space.  
- Reduced density matrix of either qubit is maximally mixed: \(\tfrac12 I\).  
- Von Neumann entanglement entropy = 1 (max for 2 qubits).

---

## 5) Creating entangled states (Bell pairs)

**Circuit for \(|\Phi^{+}\rangle\).**
1. Initialise \(|00\rangle\).  
2. Apply \(H\) to qubit 0: \(\tfrac{1}{\sqrt2}(|00\rangle+|10\rangle)\).  
3. Apply CNOT(control 0 → target 1): \(\tfrac{1}{\sqrt2}(|00\rangle+|11\rangle)\).

**Other Bell states**
- Apply \(Z\) on q0 to get \(|\Phi^{-}\rangle\).  
- Apply \(X\) on q1 after step 3 for \(|\Psi^{+}\rangle\).  
- Combine \(X\) & \(Z\) for \(|\Psi^{-}\rangle\).

**Qiskit snippet**
```python
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

bell = QuantumCircuit(2, 2)
bell.h(0)
bell.cx(0, 1)
bell.measure([0,1], [0,1])

backend = AerSimulator()
counts = backend.run(transpile(bell, backend), shots=2000).result().get_counts()
print(counts)  # ≈ {'00': 1000, '11': 1000}
```

---

## 6) Measuring correlations & violating CHSH

### Computational-basis measurement  
For \(|\Phi^{+}\rangle\): only results `00` or `11`. Perfect correlation.

### CHSH test (outline)
1. Prepare many Bell pairs.  
2. Randomly choose basis angles \(a,a',b,b'\) (e.g., Pauli-\(X\)/\(Z\) combinations).  
3. Record outcomes, compute correlation \(E\).  
4. Combine to get \(S\); quantum should yield \(\approx2.7\).

**Qiskit has a built-in example** in the textbook; see *ch‐applications/quantum‐internet/CHSH* notebook.

---

## 7) Applications

### 7.1 Quantum teleportation (three-qubit circuit)
1. Alice owns unknown qubit \(|\psi\rangle\) + half of Bell pair (\(q_1\)); Bob has \(q_2\).  
2. Alice performs Bell-basis measurement on \((|\psi\rangle,q_1)\) ⇒ 2 classical bits.  
3. Sends bits to Bob; Bob applies \(X\) / \(Z\) corrections.  
4. Bob’s qubit becomes \(|\psi\rangle\); original destroyed.

### 7.2 Super-dense coding
- Shared Bell pair.  
- Alice encodes 2 classical bits by applying \(\{I,X,Z,XZ\}\) to her half and sends it.  
- Bob performs Bell-basis measurement ⇒ recovers 2 bits.

### 7.3 Entanglement in QML
- **Feature maps**: entangling qubits embeds data into high-dimensional Hilbert space.  
- **Variational circuits**: entangled layers increase expressivity (but risk barren plateaus).  
- **Quantum GANs**: entangled generators sample complex distributions.

---

## 8) Visualising entanglement

| Tool | What it shows |
|------|---------------|
| **Statevector** | Full amplitudes. Bell pair → `[0.707, 0, 0, 0.707]`. |
| **Density matrix (`plot_state_city`)** | Off-diagonal coherence; partial trace shows mixed state. |
| **Concurrence / negativity** | Entanglement monotones (Bell states = 1). |

**Reduced density example:**  
\[
\rho_{AB}=|\Phi^{+}\rangle\langle\Phi^{+}|,\quad
\rho_A=\operatorname{Tr}_B(\rho_{AB})=\tfrac12 I.
\]
Mixed local state ⇒ global entanglement.

---

## 9) Worked algebra/examples

**Example A — Partial trace.** Compute \(\rho_A\) for \(|\Psi^{+}\rangle\); result \(=\tfrac12I\).

**Example B — CHSH value.** For measurement axes \(Z, X\) on Alice and \(Z+X, Z-X\) on Bob (properly normalised), algebra yields \(S=2\sqrt2\).

**Example C — Teleportation corrections.** Show measurement outcomes 01 → Bob applies \(X\); 10 → \(Z\); 11 → \(XZ\).

---

## 10) Mini-exercises (brief answers in Appendix)

1. **Separable or entangled?** Examine \(|\phi\rangle=\tfrac{|00\rangle+|01\rangle}{\sqrt2}\).  
2. **Create \(|\Psi^{-}\rangle\) in Qiskit using the minimal additional gate after Bell circuit.**  
3. **Density matrix check.** Calculate \(\rho_A\) for \(|00\rangle\). Explain why it’s pure.  
4. **Classical bound.** Prove \(S\le2\) for deterministic local models.  
5. **Noise impact.** What happens to \(|\Phi^{+}\rangle\) after a depolarising channel with probability \(p\)?  
6. **Teleportation fidelity.** Compute average fidelity if classical bits are flipped with probability \(0.1\).

---

## 11) FAQ / common pitfalls

- **“Does entanglement send information faster than light?”** No. Correlations appear instantaneously but require classical side-channel for usable communication.  
- **“Why do I sometimes see `'01'` in my Bell counts?”** Finite sampling + gate noise; increase shots and check device error rates.  
- **“Can I visualise a 3-qubit Bloch sphere?”** Not directly—use statevector amplitudes or reduced single-qubit Bloch vectors.  
- **“Teleportation seems to copy the state.”** Original is destroyed by Alice’s Bell measurement, respecting no-cloning.

---

## 12) Summary (Session 3)

- Entanglement = non-separable multi-qubit correlations; key quantum resource.  
- Bell states form a maximally entangled basis; created with \(H\)+CNOT.  
- CHSH inequality quantifies non-locality; experiments violate classical bound.  
- Teleportation moves quantum states via classical bits + entanglement; super-dense coding doubles classical capacity.  
- Entanglement powers many quantum-AI primitives but suffers from noise on NISQ hardware.

---

## 13) Looking ahead

- **Next Session:** Quantum Query Algorithms (Grover, amplitude amplification).  
- **Homework reminder (HW 1 extension):**  
  - Implement \(|\Psi^{-}\rangle\) & verify correlations.  
  - Run the provided teleportation notebook; answer reflection questions. (Ungraded but strongly recommended.)

---

## 14) Appendix — solutions to mini-exercises

1. \(|\phi\rangle=\tfrac{|0\rangle(|0\rangle+|1\rangle)}{\sqrt2}\) → separable (rank 1).  
2. Apply \(X\) to *control* qubit after Bell circuit or \(Z\) to target then \(X\) etc.; minimal: `bell.x(0); bell.z(0)` produces \(|\Psi^{-}\rangle\).  
3. \(|00\rangle\) is product state ⇒ \(\rho_A=\begin{pmatrix}1&0\\0&0\end{pmatrix}\); pure (trace = 1, \(\mathrm{Tr}\rho^2=1\)).  
4. Enumerate deterministic outcomes ±1; linearity over shared randomness ⇒ \(S\le2\).  
5. Depolarising channel: \(\rho\to(1-p)\rho + p\,\tfrac14 I\); concurrence drops linearly with \(1-p\).  
6. Average teleportation fidelity \(F=1-\frac{4}{3}p_{\text{flip}}=1-\frac{4}{3}\cdot0.1\approx0.867\).

